In [1]:
import os
import glob
from Bio import SeqIO
from tqdm.auto import tqdm
import pylev
import matplotlib.pyplot as plt
import numpy as np
import copy

In [116]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
def print_alignment(seq1,seq2):
    alignment = pairwise2.align.globalms(seq1, seq2,2,-1,-8,-0.3)#gap penalty of -5 seems to work well at preventing them from forming!
    al1 = alignment[0][0]
    al2 = alignment[0][1]

    i = 0
    print("Alignment:")
    while(i <= len(al1)):
        sub_al1 = al1[i:min(len(al1),i+70)]
        sub_al2 = al2[i:min(len(al2),i+70)]
        
        print(f"{i}-{min(len(al1),i+70)}")
        print(sub_al1)
        print("".join(["|" if sub_al1[j] == sub_al2[j] else " " if sub_al1[j] == "-" or sub_al2[j] == "-" else "." for j in range(0,len(sub_al1))]))
        print(sub_al2)
        print()
        i += 70
    
    gap_percentage = ( len(al1)-max(len(seq1),len(seq2)) )/len(al1)*100
    print(f"Gap percentage: {gap_percentage}")
    

## Load the ASR file from space separated file into something useful

In [122]:
import csv
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

ASR_filename = "../mock_data/ASR_protein.txt"
HuERb_filename = "../mock_data/HuERb.fa"
ApER_filename = "../mock_data/ApER.fa"

raw_seq = ""
reader = csv.reader(open(ASR_filename), delimiter=" ")
for row in reader:
    raw_seq += row[1]
asr_seq = Seq(raw_seq)

HuERb_seq = next(SeqIO.parse(HuERb_filename,"fasta")).seq
ApER_seq = next(SeqIO.parse(ApER_filename,"fasta")).seq

# print(HuERb_seq[128:-27])
# print(ApER_seq[76:-83])
# print(asr_seq)
print("\nApER_seq,HuERb_seq")
print_alignment(ApER_seq,HuERb_seq)
print("\nApER_seq,asr_seq")
print_alignment(ApER_seq,asr_seq)
print("\nHuERb_seq,asr_seq")
print_alignment(HuERb_seq,asr_seq)



ApER_seq,HuERb_seq
Alignment:
0-70
M---VSPSPCMPVLPLSGAGGGGEGTAGS-----------AHLQAATSTTPEAGSGTLPHNFLMDVHGPG
|   .|||...................||           ....|.|...|.......|.|......|||
MDIKNSPSSLNSPSSYNCSQSILPLEHGSIYIPSSYVDSHHEYPAMTFYSPAVMNYSIPSNVTNLEGGPG

70-140
TLGASP---------------------------------------------------SSHMKQEVGGAVP
....||                                                   .|..........|
RQTTSPNVLWPTPGHLSPLVVHRQLSHLYAEPQKSPWCEARSLEHTLPVNRETLKRKVSGNRCASPVTGP

140-210
GSSIQAKLCQVCSDNASGFHYGVWSCEGCKAFFKRSIQGPVDYICPATNTCTIDKHRRKSCQACRLRRCY
||...|..|.||||.|||.||||||||||||||||||||..||||||||.|||||.|||||||||||.||
GSKRDAHFCAVCSDYASGYHYGVWSCEGCKAFFKRSIQGHNDYICPATNQCTIDKNRRKSCQACRLRKCY

210-280
EVGMNKGSQR-----------------------KEKRNSGNTSSLKGKRCRADSSDSAVNSTNNGASSSK
||||.|...|                       |.||..|...........|.|....|           
EVGMVKCGSRRERCGYRLVRRQRSADEQLHCAGKAKRSGGHAPRVRELLLDALSPEQLV-----------

280-350
SSKRSRSASILEALQKADLPVMDSYHNHNLPATRVHLLNTLIKLADRELVYLINWAKHVPGYTCLTLGDQ
      

In [119]:
HuERb_MSA_filename = "../mock_data/HuERb_MSA.txt"
ApER_MSA_filename = "../mock_data/ApER_MSA.txt"

HuERb_MSA_seq = Seq("")
ApER_MSA_seq = Seq("")

with open(HuERb_MSA_filename) as file:
    line = [line.rstrip() for line in file][0]
    line = line.replace("-","*")
    HuERb_MSA_seq = Seq(line)
    
with open(ApER_MSA_filename) as file:
    line = [line.rstrip() for line in file][0]
    line = line.replace("-","*")
    ApER_MSA_seq = Seq(line)

print_alignment(HuERb_MSA_seq,asr_seq)
print_alignment(ApER_MSA_seq,asr_seq)

Alignment:
0-70
*SGNRCASPVTG*PGSKRDAHFCAVCSDYASGYHYGVWSCEGCKAFFKRSIQ---------GHNDYICPA
                      |.||.|.|||||||||||||||||||||||         ||.||.|||
----------------------CLVCGDNASGYHYGVWSCEGCKAFFKRSIQGWCARQNTDGHIDYMCPA

70-140
TNQCTIDKNRRKSCQACRLRKCYEVGMVKCGSRRERCGY*RLVRRQR*****************SADEQL
||.|||||.|||||||||||||.||||                                           
TNECTIDKHRRKSCQACRLRKCLEVGM-------------------------------------------

140-210
HCAGKAKRSG****GHAP*****RVRELLLDALSPEQ*******LVLTLLEAEPPHVLISRPSA******
                               ..|.||       |...||||||..|..|          
-------------------------------TPSAEQ-------LISALLEAEPETVYAS----------

210-280
********PFTEAS************************MMMSLTKLADKELVHMISWAKKIPGFVELSLF
   .....|.||.|                        |...||.|||.||||.|.||||||||..|||.
---HDPDKPDTEVS------------------------MLTTLTNLADRELVHIINWAKKIPGFTDLSLG

280-350
DQVRLLESCWMEVLMMGLMWRSIDHPGK***LIFAPDLVLDRDEGKCVEGILEIFDMLLATTSRFRELKL
||..||..||||.|..||.|||||| 